In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.color import rgb2gray
from skimage import img_as_ubyte

# Tính histogram của ảnh (kênh độ sáng hoặc grayscale)
def calculate_histogram(image):
    """
    Tính histogram của ảnh (kênh độ sáng hoặc grayscale).
    """
    if len(image.shape) == 3:  # Nếu ảnh màu, chuyển sang grayscale
        gray_image = rgb2gray(image)
    else:
        gray_image = image

    # Tính histogram với 256 bins (từ 0 đến 1 cho ảnh grayscale)
    histogram, _ = np.histogram(gray_image, bins=256, range=(0, 1))
    
    return histogram, gray_image

# Tính CDF từ histogram
def calculate_cdf(histogram):
    """
    Tính CDF từ histogram.
    """
    cdf = histogram.cumsum()  # Cộng dồn histogram
    cdf_normalized = cdf / cdf[-1]  # Chuẩn hóa cdf về khoảng [0, 1]
    
    return cdf_normalized

# Vẽ histogram và CDF của ảnh
def plot_histogram_and_cdf(image):
    """
    Vẽ histogram và CDF của ảnh.
    """
    histogram, gray_image = calculate_histogram(image)
    cdf_normalized = calculate_cdf(histogram)
    
    # Vẽ histogram và CDF
    plt.figure(figsize=(10, 5))
    
    # Vẽ Histogram
    plt.subplot(1, 2, 1)
    plt.title("Histogram")
    plt.plot(histogram, color='black')
    plt.xlabel("Intensity")
    plt.ylabel("Frequency")
    
    # Vẽ CDF
    plt.subplot(1, 2, 2)
    plt.title("CDF")
    plt.plot(cdf_normalized, color='blue')
    plt.xlabel("Intensity")
    plt.ylabel("Cumulative Probability")
    
    plt.tight_layout()
    plt.show()

# Tính độ sáng trung bình của ảnh
def calculate_average_brightness(image):
    """
    Tính độ sáng trung bình của ảnh (hoặc kênh độ sáng trong ảnh màu).
    """
    if len(image.shape) == 3:  # Nếu ảnh màu, chuyển sang grayscale
        gray_image = rgb2gray(image)
    else:
        gray_image = image
    
    avg_brightness = np.mean(gray_image)  # Tính trung bình của các giá trị pixel
    return avg_brightness

# Tính độ sáng trung bình có trọng số
def calculate_weighted_brightness(image):
    """
    Tính độ sáng trung bình có trọng số (weighted average brightness) của ảnh.
    """
    if len(image.shape) == 3:
        gray_image = rgb2gray(image)
    else:
        gray_image = image

    # Trọng số cho các pixel (có thể dùng các thông số như vị trí pixel)
    height, width = gray_image.shape
    y, x = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
    
    # Tính trọng số (ví dụ, trọng số tăng dần từ góc ảnh vào trung tâm)
    distance = np.sqrt((x - width / 2)**2 + (y - height / 2)**2)
    weight = np.exp(-distance / (0.1 * max(height, width)))

    # Tính trung bình có trọng số
    weighted_avg_brightness = np.sum(gray_image * weight) / np.sum(weight)
    return weighted_avg_brightness

# Tính độ tương phản của ảnh
def calculate_contrast(image):
    """
    Tính độ tương phản của ảnh dựa trên độ lệch chuẩn của kênh độ sáng.
    """
    if len(image.shape) == 3:
        gray_image = rgb2gray(image)
    else:
        gray_image = image

    contrast = np.std(gray_image)  # Độ lệch chuẩn
    return contrast

# Hàm đánh giá tất cả các ảnh trong thư mục
def evaluate_all_images(root_directory):
    # Kiểm tra thư mục gốc tồn tại
    if not os.path.exists(root_directory):
        print(f"Directory {root_directory} does not exist.")
        return

    # Lấy danh sách tất cả các thư mục con (train, test, val)
    subfolders = ['train/images', 'test/images', 'val/images']
    total_histograms = []
    total_cdfs = []
    total_avg_brightness = []
    total_weighted_brightness = []
    total_contrast = []
    num_images = 0

    # Duyệt qua từng thư mục con
    for subfolder in subfolders:
        folder_path = os.path.join(root_directory, subfolder)

        if not os.path.exists(folder_path):
            print(f"Subfolder {subfolder} does not exist in {root_directory}.")
            continue

        # Lấy danh sách ảnh trong thư mục
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Đánh giá từng ảnh
        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)

            try:
                # Đọc ảnh
                img = io.imread(image_path)

                # Tính Histogram và CDF cho ảnh
                histogram, gray_image = calculate_histogram(img)
                cdf_normalized = calculate_cdf(histogram)

                # Lưu lại histogram và CDF
                total_histograms.append(histogram)
                total_cdfs.append(cdf_normalized)

                # Tính độ sáng trung bình
                avg_brightness = calculate_average_brightness(img)
                total_avg_brightness.append(avg_brightness)

                # Tính độ sáng có trọng số
                weighted_brightness = calculate_weighted_brightness(img)
                total_weighted_brightness.append(weighted_brightness)

                # Tính độ tương phản
                contrast = calculate_contrast(img)
                total_contrast.append(contrast)

                # Vẽ Histogram và CDF cho mỗi ảnh (tuỳ chọn)
                #plot_histogram_and_cdf(img)

                num_images += 1

            except Exception as e:
                print(f"Error processing {image_file}: {e}")

    # Tính trung bình các giá trị
    avg_brightness = np.mean(total_avg_brightness) if total_avg_brightness else None
    avg_weighted_brightness = np.mean(total_weighted_brightness) if total_weighted_brightness else None
    avg_contrast = np.mean(total_contrast) if total_contrast else None

    print("\nAverage Scores:")
    if avg_brightness is not None:
        print(f"Average Brightness: {avg_brightness:.2f}")
    if avg_weighted_brightness is not None:
        print(f"Average Weighted Brightness: {avg_weighted_brightness:.2f}")
    if avg_contrast is not None:
        print(f"Average Contrast: {avg_contrast:.2f}")
    
    return avg_brightness, avg_weighted_brightness, avg_contrast

# Define root directories for datasets
root_directories = [
    "/kaggle/input/cs406-data/Dark_Dataset/Dark_Dataset",
    "/kaggle/input/cs406-data/Enhanced_Dataset/Enhanced_Dataset",
    "/kaggle/input/cs406-data/enhance_dce/enhance_dce",
]

# Chạy đánh giá cho từng tập dữ liệu
for root_directory in root_directories:
    print(f"\nEvaluating dataset in: {root_directory}")
    avg_brightness, avg_weighted_brightness, avg_contrast = evaluate_all_images(root_directory)



Evaluating dataset in: /kaggle/input/cs406-data/Dark_Dataset/Dark_Dataset

Average Scores:
Average Brightness: 0.03
Average Weighted Brightness: 0.04
Average Contrast: 0.03

Evaluating dataset in: /kaggle/input/cs406-data/Enhanced_Dataset/Enhanced_Dataset

Average Scores:
Average Brightness: 0.13
Average Weighted Brightness: 0.16
Average Contrast: 0.11

Evaluating dataset in: /kaggle/input/cs406-data/enhance_dce/enhance_dce

Average Scores:
Average Brightness: 0.38
Average Weighted Brightness: 0.44
Average Contrast: 0.22
